# Get winrate of each champion

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col

In [2]:
spark = SparkSession.builder.appName("ReadData").getOrCreate()

In [3]:
file_path = "D:/BigData/Data/match_detail.json"
df = spark.read.json(file_path)

In [4]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- info: struct (nullable = true)
 |    |-- gameCreation: long (nullable = true)
 |    |-- gameDuration: long (nullable = true)
 |    |-- gameEndTimestamp: long (nullable = true)
 |    |-- gameId: long (nullable = true)
 |    |-- gameMode: string (nullable = true)
 |    |-- gameName: string (nullable = true)
 |    |-- gameStartTimestamp: long (nullable = true)
 |    |-- gameType: string (nullable = true)
 |    |-- gameVersion: string (nullable = true)
 |    |-- mapId: long (nullable = true)
 |    |-- participants: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- allInPings: long (nullable = true)
 |    |    |    |-- assistMePings: long (nullable = true)
 |    |    |    |-- assists: long (nullable = true)
 |    |    |    |-- baitPings: long (nullable = true)
 |    |    |    |-- baronKills: long (nullable = true)
 |    |    |    |-- basicPings: long (nullable = true)
 |    |    |    |-- 

In [5]:
df.count()

4065

In [6]:
df.show()

+---------------+--------------------+--------------------+
|_corrupt_record|                info|            metadata|
+---------------+--------------------+--------------------+
|           NULL|{1702218495729, 1...|{2, VN2_263621944...|
|           NULL|{1702214839655, 1...|{2, VN2_263529508...|
|           NULL|{1702213456722, 1...|{2, VN2_263503073...|
|           NULL|{1702212129650, 1...|{2, VN2_263457862...|
|           NULL|{1702210448240, 1...|{2, VN2_263425557...|
|           NULL|{1702208898115, 1...|{2, VN2_263406060...|
|           NULL|{1702204199914, 1...|{2, VN2_263311776...|
|           NULL|{1702141622861, 9...|{2, VN2_262473336...|
|           NULL|{1702132388766, 1...|{2, VN2_262242252...|
|           NULL|{1702130960818, 1...|{2, VN2_262199389...|
|           NULL|{1702222055120, 2...|{2, VN2_263714400...|
|           NULL|{1702219444484, 1...|{2, VN2_263646609...|
|           NULL|{1702199204289, 9...|{2, VN2_263218782...|
|           NULL|{1702198173404, 8...|{2

In [7]:
match_df = df.select(
    explode("info.participants").alias("participant"),
    "info.gameId",
    "info.gameMode",
    "info.gameType",
    "info.gameVersion",
    ).select(
    "gameId",
    "gameMode",
    "participant.riotIdGameName",
    "participant.championId",
    "participant.championName",
    "participant.lane",
    "participant.teamId",
    "participant.teamPosition",
    "participant.win",
).where("gameMode = 'CLASSIC'")

In [8]:
match_df.show(20)

+---------+--------+---------------+----------+------------+------+------+------------+-----+
|   gameId|gameMode| riotIdGameName|championId|championName|  lane|teamId|teamPosition|  win|
+---------+--------+---------------+----------+------------+------+------+------------+-----+
|263311776| CLASSIC|          Kenbu|       114|       Fiora|   TOP|   100|         TOP| true|
|263311776| CLASSIC|GOOD GAME GG XD|        64|      LeeSin|JUNGLE|   100|      JUNGLE| true|
|263311776| CLASSIC|   tennaymanman|         8|    Vladimir|MIDDLE|   100|      MIDDLE| true|
|263311776| CLASSIC|  not look back|       145|       Kaisa|BOTTOM|   100|      BOTTOM| true|
|263311776| CLASSIC|  jonthelonewof|       497|       Rakan|BOTTOM|   100|     UTILITY| true|
|263311776| CLASSIC|        Quankun|       777|        Yone|   TOP|   200|         TOP|false|
|263311776| CLASSIC| Yêu Thiên Bình|        56|    Nocturne|JUNGLE|   200|      JUNGLE|false|
|263311776| CLASSIC|     Kẽ Vẽ Mộng|         3|       Galio|

In [9]:
match_df = match_df.withColumn("win", col("win").cast("integer"))


In [10]:
match_df.show(100)

+---------+--------+----------------+----------+------------+------+------+------------+---+
|   gameId|gameMode|  riotIdGameName|championId|championName|  lane|teamId|teamPosition|win|
+---------+--------+----------------+----------+------------+------+------+------------+---+
|263311776| CLASSIC|           Kenbu|       114|       Fiora|   TOP|   100|         TOP|  1|
|263311776| CLASSIC| GOOD GAME GG XD|        64|      LeeSin|JUNGLE|   100|      JUNGLE|  1|
|263311776| CLASSIC|    tennaymanman|         8|    Vladimir|MIDDLE|   100|      MIDDLE|  1|
|263311776| CLASSIC|   not look back|       145|       Kaisa|BOTTOM|   100|      BOTTOM|  1|
|263311776| CLASSIC|   jonthelonewof|       497|       Rakan|BOTTOM|   100|     UTILITY|  1|
|263311776| CLASSIC|         Quankun|       777|        Yone|   TOP|   200|         TOP|  0|
|263311776| CLASSIC|  Yêu Thiên Bình|        56|    Nocturne|JUNGLE|   200|      JUNGLE|  0|
|263311776| CLASSIC|      Kẽ Vẽ Mộng|         3|       Galio|MIDDLE|  

In [11]:
from pyspark.sql.functions import col, count, sum
win_rate_df = match_df.groupBy("championName") \
    .agg(
        sum("win").alias("totalWins"), 
        count("win").alias("totalGames")
    ) \
    .withColumn("winRate", col("totalWins") / col("totalGames")) \
    .orderBy("championName")

In [12]:
win_rate_df.show(200)

+------------+---------+----------+-------------------+
|championName|totalWins|totalGames|            winRate|
+------------+---------+----------+-------------------+
|      Aatrox|      239|       493| 0.4847870182555781|
|        Ahri|       50|       145| 0.3448275862068966|
|       Akali|      202|       397| 0.5088161209068011|
|      Akshan|       34|        51| 0.6666666666666666|
|     Alistar|       79|       173|0.45664739884393063|
|       Amumu|        8|        12| 0.6666666666666666|
|      Anivia|       27|        44| 0.6136363636363636|
|       Annie|       14|        35|                0.4|
|    Aphelios|       94|       189| 0.4973544973544973|
|        Ashe|      130|       273|0.47619047619047616|
| AurelionSol|       19|        52|0.36538461538461536|
|        Azir|       38|        91| 0.4175824175824176|
|        Bard|       37|        93| 0.3978494623655914|
|     Belveth|       76|       113|  0.672566371681416|
|  Blitzcrank|       83|       169| 0.4911242603

In [22]:
match_df.select("*").where("teamPosition is NULL").show()

+------+--------+--------------+----------+------------+----+------+------------+---+
|gameId|gameMode|riotIdGameName|championId|championName|lane|teamId|teamPosition|win|
+------+--------+--------------+----------+------------+----+------+------------+---+
+------+--------+--------------+----------+------------+----+------+------------+---+



In [14]:
match_df_unique = match_df.dropDuplicates()

In [15]:
match_df_unique.count()

17950

In [16]:
match_df.count()

23500

In [17]:
win_rate_unique_df = match_df_unique.groupBy("championName") \
    .agg(
        sum("win").alias("totalWins"), 
        count("win").alias("totalGames")
    ) \
    .withColumn("winRate", col("totalWins") / col("totalGames")) \
    .orderBy("championName")

In [18]:
win_rate_unique_df.show()

+------------+---------+----------+-------------------+
|championName|totalWins|totalGames|            winRate|
+------------+---------+----------+-------------------+
|      Aatrox|      194|       403| 0.4813895781637717|
|        Ahri|       44|       120|0.36666666666666664|
|       Akali|      151|       303|0.49834983498349833|
|      Akshan|       23|        37| 0.6216216216216216|
|     Alistar|       54|       123|0.43902439024390244|
|       Amumu|        7|        11| 0.6363636363636364|
|      Anivia|       16|        32|                0.5|
|       Annie|       10|        24| 0.4166666666666667|
|    Aphelios|       67|       136|0.49264705882352944|
|        Ashe|      108|       214| 0.5046728971962616|
| AurelionSol|       15|        37|0.40540540540540543|
|        Azir|       31|        71|0.43661971830985913|
|        Bard|       24|        61|0.39344262295081966|
|     Belveth|       49|        79|  0.620253164556962|
|  Blitzcrank|       67|       135| 0.4962962962

In [19]:
win_rate_df.show()

+------------+---------+----------+-------------------+
|championName|totalWins|totalGames|            winRate|
+------------+---------+----------+-------------------+
|      Aatrox|      239|       493| 0.4847870182555781|
|        Ahri|       50|       145| 0.3448275862068966|
|       Akali|      202|       397| 0.5088161209068011|
|      Akshan|       34|        51| 0.6666666666666666|
|     Alistar|       79|       173|0.45664739884393063|
|       Amumu|        8|        12| 0.6666666666666666|
|      Anivia|       27|        44| 0.6136363636363636|
|       Annie|       14|        35|                0.4|
|    Aphelios|       94|       189| 0.4973544973544973|
|        Ashe|      130|       273|0.47619047619047616|
| AurelionSol|       19|        52|0.36538461538461536|
|        Azir|       38|        91| 0.4175824175824176|
|        Bard|       37|        93| 0.3978494623655914|
|     Belveth|       76|       113|  0.672566371681416|
|  Blitzcrank|       83|       169| 0.4911242603

# GET THE WIN RATE OF MATCHUP

In [24]:
match_df_unique.createOrReplaceTempView("match")

In [29]:
join_query = """
SELECT m1.gameId, m1.championName as champion1, m2.championName as champion2,m1.teamPosition, m2.teamPosition,m1.teamId as team1, m2.teamId as team2, m1.win
FROM match m1, match m2
WHERE m1.gameId = m2.gameId 
    AND m1.teamPosition = m2.teamPosition
    AND m1.teamId != m2.teamId
"""

In [30]:
matchup_each_game_df = spark.sql(join_query)

In [31]:
matchup_each_game_df.show()

+---------+-----------+----------+------------+------------+-----+-----+---+
|   gameId|  champion1| champion2|teamPosition|teamPosition|team1|team2|win|
+---------+-----------+----------+------------+------------+-----+-----+---+
|260001330|       Sona|Blitzcrank|     UTILITY|     UTILITY|  100|  200|  0|
|261519000|    Alistar|      Pyke|     UTILITY|     UTILITY|  100|  200|  0|
|263905840|       Hwei|     Brand|     UTILITY|     UTILITY|  100|  200|  0|
|263161124|     Xerath|      Zyra|     UTILITY|     UTILITY|  100|  200|  0|
|260785481|     Graves|   Kindred|      JUNGLE|      JUNGLE|  100|  200|  1|
|262101218|MissFortune|      Lulu|     UTILITY|     UTILITY|  200|  100|  0|
|260339888|       Jinx|     Kaisa|      BOTTOM|      BOTTOM|  200|  100|  0|
|261754134|       Nami|Blitzcrank|     UTILITY|     UTILITY|  100|  200|  1|
|261405569| Blitzcrank|    Twitch|     UTILITY|     UTILITY|  200|  100|  0|
|248821392|        Zed|  Vladimir|      MIDDLE|      MIDDLE|  100|  200|  1|

In [41]:
matchup_df = matchup_each_game_df.groupBy('champion1', 'champion2').agg(
    sum('win').alias('total_wins'),
    count('gameId').alias('total_games')
).withColumn('champion1_win_rate', col('total_wins') / col('total_games')).orderBy('champion1', 'champion2')

In [42]:
matchup_df.show()

+---------+------------+----------+-----------+-------------------+
|champion1|   champion2|total_wins|total_games| champion1_win_rate|
+---------+------------+----------+-----------+-------------------+
|   Aatrox|      Aatrox|         1|          2|                0.5|
|   Aatrox|       Akali|         8|         16|                0.5|
|   Aatrox|     Camille|         4|          9| 0.4444444444444444|
|   Aatrox|  Cassiopeia|         1|          1|                1.0|
|   Aatrox|      Darius|         5|         12| 0.4166666666666667|
|   Aatrox|     DrMundo|         1|          1|                1.0|
|   Aatrox|      Draven|         1|          2|                0.5|
|   Aatrox|       Fiora|         6|         17|0.35294117647058826|
|   Aatrox|        Fizz|         0|          1|                0.0|
|   Aatrox|   Gangplank|         4|          7| 0.5714285714285714|
|   Aatrox|       Garen|         2|          3| 0.6666666666666666|
|   Aatrox|        Gnar|         4|          6| 

In [47]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
matchups_over_30_games = matchup_df.filter(F.col('total_games') > 30)
windowSpec = Window.partitionBy('champion1').orderBy('champion1_win_rate')
ranked_matchups = matchups_over_30_games.withColumn("rank", F.row_number().over(windowSpec))
worst_matchups = ranked_matchups.filter(F.col("rank") <= 5)
worst_matchups.show()

+---------+---------+----------+-----------+-------------------+----+
|champion1|champion2|total_wins|total_games| champion1_win_rate|rank|
+---------+---------+----------+-----------+-------------------+----+
|   Aatrox|     Yone|        17|         39| 0.4358974358974359|   1|
|   Aatrox|    Jayce|        19|         39|0.48717948717948717|   2|
|   Aatrox|      Jax|        19|         34| 0.5588235294117647|   3|
| Aphelios|    Kaisa|        13|         31|0.41935483870967744|   1|
|  Caitlyn|   Ezreal|        15|         32|            0.46875|   1|
|   Ezreal|    Kaisa|        61|        121| 0.5041322314049587|   1|
|   Ezreal|  Caitlyn|        17|         32|            0.53125|   2|
|   Ezreal|    Varus|        18|         31| 0.5806451612903226|   3|
|   Graves|   Khazix|        19|         38|                0.5|   1|
|   Graves|   LeeSin|        53|         97| 0.5463917525773195|   2|
| JarvanIV|   LeeSin|        16|         41| 0.3902439024390244|   1|
|      Jax|   Aatrox